# Data Processing <sub>(using PySpark)</sub>

## Create Spark Session

In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Read the Non-Processed Data from CSV

In [21]:
df = spark.read.csv('lexicon_data/non_processed_data.csv', header=True)

In [22]:
df.show()

+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|   vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|      ma|          2|          NULL|                NULL|                NULL|                NULL|                NULL|
|     sud|          3|          NULL|                NULL|                NULL|                NULL|                NULL|
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|Definisi|          8|          NULL|                NULL|                NULL|                NULL|                NULL|
|  tujuan|          6|  

## Convert all the Records in `vocab` Column to Lowercase

In [23]:
from pyspark.sql.functions import *

# convert the data in vocab column to lowercase
df = df.withColumn(
    'vocab',
    lower(col('vocab'))
)

df.show()

+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|   vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|      ma|          2|          NULL|                NULL|                NULL|                NULL|                NULL|
|     sud|          3|          NULL|                NULL|                NULL|                NULL|                NULL|
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|definisi|          8|          NULL|                NULL|                NULL|                NULL|                NULL|
|  tujuan|          6|  

## Study Anomaly in `part_of_speech` Column

In [24]:
df.select('part_of_speech').distinct().show()

+--------------------+
|      part_of_speech|
+--------------------+
|sedap, lazat, ser...|
|empiris, saintifi...|
|segalanya, sekali...|
|bernafas, tumbuh,...|
|hal, perkara, per...|
|             adverba|
|           adjektif:|
|bersungguh-sunggu...|
|            adjektif|
|kemampuan, keupay...|
|       sinar, kilau,|
|           adjektif,|
|          kata nama,|
|had, takat, sempa...|
|sudah lazim, buka...|
|        tidak sampai|
|           kata nama|
|          kata tugas|
|                sisa|
|          kata kerja|
+--------------------+
only showing top 20 rows



In here, it is found that
- there exist records that are not part of speech
- some proper part of speech has punctuation joined behind

Hence, to standardize with the data, the following cleansing processes are done.

### Remove Punctuation from the Columns

In [25]:
df = df.withColumn(
    'part_of_speech',  
    regexp_replace(col('part_of_speech'), "[^\w\s]", "") 
)

df.select('part_of_speech').distinct().show()

+--------------------+
|      part_of_speech|
+--------------------+
|segalanya sekalia...|
|empiris saintifik...|
|kemampuan keupaya...|
|             adverba|
|            adjektif|
|hal perkara peris...|
|sudah lazim bukan...|
|  had takat sempadan|
|         sinar kilau|
|bernafas tumbuh m...|
|        tidak sampai|
|           kata nama|
|          kata tugas|
|sedap lazat seron...|
|bersungguhsungguh...|
|                sisa|
|          kata kerja|
|                NULL|
+--------------------+



### Set the Values that are not Part of Speech to NULL

the allowed part of speech values include _kata nama_ (noun), _kata tugas_ or _kata kerja_ (verb), _adjektif_ (adjective), and _adverba_ (adverb).

In [26]:
allowed_pos = ['kata nama', 'kata tugas', 'kata kerja', 'adjektif', 'adverba']

# remove the data that is not considered part of speech
df = df.withColumn(
    'part_of_speech',
    when(col('part_of_speech').isin(allowed_pos), col('part_of_speech')).otherwise(None)
)

df.select('part_of_speech').distinct().show()
df.show()

+--------------+
|part_of_speech|
+--------------+
|       adverba|
|      adjektif|
|     kata nama|
|    kata tugas|
|    kata kerja|
|          NULL|
+--------------+

+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|   vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|      ma|          2|          NULL|                NULL|                NULL|                NULL|                NULL|
|     sud|          3|          NULL|                NULL|                NULL|                NULL|                NULL|
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr.

## Drop Duplicated Records

In [27]:
# Drop duplicated records
df = df.dropDuplicates()
df.show()
print("Number of Unique Record:", df.count())

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      perlu|          5|    kata tugas|1. keharusan (yg ...|1. sembahyang dan...|mesti;wajib;harus...|memerlukan;keperluan|
|      waktu|          5|     kata nama|1. rangkaian atau...|1. secara rohania...|masa;jangka masa;...|sewaktu;sewaktu-w...|
|     mundur|          6|      adjektif|1. (berjalan, ber...|1. dia mundur set...|merosot;meleset;m...|memundurkan;kemun...|
|      sifat|          5|     kata nama|1. keadaan atau r...|1. adapun sifatny...|keadaan;rupa;tand...|bersifat;menyifatkan|
|       suci|          4|      adjektif|1. bersih (dr seg...|1. makanan yg suc...|bersih;murni;tida...|    bersuci;kesucian|


## Check with the Vocabs that Definition is Undefined

In [28]:
# check the records with no definition scrapped
null_def_df = df.filter(col('definitions').isNull())

null_def_df.show()
print("Number of Unique Record with No Definition:", null_def_df.count())

+-------------+-----------+--------------+-----------+------------+--------+--------------------+
|        vocab|word_length|part_of_speech|definitions|sentences_eg|synonyms|       derived_words|
+-------------+-----------+--------------+-----------+------------+--------+--------------------+
|         agar|          4|          NULL|       NULL|        NULL|    NULL|                NULL|
|        dewan|          5|          NULL|       NULL|        NULL|    NULL|                NULL|
|        harta|          5|     kata nama|       NULL|        NULL|    NULL|            berharta|
|      diatasi|          7|          NULL|       NULL|        NULL|    NULL|                NULL|
|    keupayaan|          9|     kata nama|       NULL|        NULL|    NULL|seupaya-upayanya;...|
|   penindasan|         10|          NULL|       NULL|        NULL|    NULL|                NULL|
|     rakannya|          8|          NULL|       NULL|        NULL|    NULL|                NULL|
|  mengajarkan|     

## Redefine the Records to be Kept and Removed

The rules are described as follows: <br>

<table>
    <thead>
        <td><b>Steps</b></td>
        <td><b>Description</b></td>
    </thead>
    <tr>
        <td style="text-align:right;"><b>1</b></td>
        <td>The words with and with no definition are separated</td>
    </tr>
    <tr>
        <td style="text-align:right; vertical-align:top;"><b>2</b></td>
        <td>
            For all the word with no definition:
            <ul>
                <li>If the word is existed in the <font style="font-family:monospace;">derived_words</font> of the words with definition, then <b>keep</b></li>
                <li>If the word is not existed in the <font style="font-family:monospace;">derived_words</font> of the words with definition, then <b>delete</b></li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align:right;"><b>3</b></td>
        <td>Combine the two dataframes (with definition and with no definition)</td>
    </tr>
</table>

The code segment of so processing is as follows.

In [29]:
# Split the DataFrame into two parts: with definition and without definition
null_df = df.filter(col("definitions").isNull())
not_null_df = df.filter(col("definitions").isNotNull())

# If vocab is a derived_words of other vocab even if it does not have definition, keep.
# Else (no definition and is not derived_words of others), delete.
matched_df = null_df.alias("null_df").join(
    not_null_df.alias("not_null_df"), 
    expr("instr(null_df.derived_words, not_null_df.vocab) > 0"), 
    "inner" 
).select("null_df.*")

df = not_null_df.union(matched_df)
df.show()
print("Number of Records after Definitions Cleansing:", df.count())

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      perlu|          5|    kata tugas|1. keharusan (yg ...|1. sembahyang dan...|mesti;wajib;harus...|memerlukan;keperluan|
|      waktu|          5|     kata nama|1. rangkaian atau...|1. secara rohania...|masa;jangka masa;...|sewaktu;sewaktu-w...|
|     mundur|          6|      adjektif|1. (berjalan, ber...|1. dia mundur set...|merosot;meleset;m...|memundurkan;kemun...|
|      sifat|          5|     kata nama|1. keadaan atau r...|1. adapun sifatny...|keadaan;rupa;tand...|bersifat;menyifatkan|
|       suci|          4|      adjektif|1. bersih (dr seg...|1. makanan yg suc...|bersih;murni;tida...|    bersuci;kesucian|


After this steps, there remained only **227 records**.

### Check again with the Duplicated Records

In [30]:
# check if there is duplicated values
print('Distinct Record Count:', df.distinct().count())

Distinct Record Count: 221


We found that there are **221 distinct records**, which unmatched with the result of previous step.

Hence, we drop again duplicated records, and verify the data count.

In [31]:
df = df.dropDuplicates()
df.show()
print('Number of Unique Records:', df.count())

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      perlu|          5|    kata tugas|1. keharusan (yg ...|1. sembahyang dan...|mesti;wajib;harus...|memerlukan;keperluan|
|      waktu|          5|     kata nama|1. rangkaian atau...|1. secara rohania...|masa;jangka masa;...|sewaktu;sewaktu-w...|
|     mundur|          6|      adjektif|1. (berjalan, ber...|1. dia mundur set...|merosot;meleset;m...|memundurkan;kemun...|
|      sifat|          5|     kata nama|1. keadaan atau r...|1. adapun sifatny...|keadaan;rupa;tand...|bersifat;menyifatkan|
|       suci|          4|      adjektif|1. bersih (dr seg...|1. makanan yg suc...|bersih;murni;tida...|    bersuci;kesucian|


## Remove Duplicated Substring in Definitions and Sentences Example

In [32]:
import re

df_rows = df.collect()

vocabs = []
for row in df_rows:
    vocab = row[0]

    vocabs.append(vocab)
    
new_def = []
for row in df_rows:
    old_def = row[3]

    if old_def != None:
        while True:
            old_def = old_def.replace(';', '; ')
            out = re.sub(r'(?<!\S)(\S+(?:\s\S+)*)\s+\1(?!\S)', '\\1', old_def)
            if out == old_def:
                break
            old_def = out
            
    new_def.append(old_def)
    
new_sentences_eg = []
for row in df_rows:
    old_sentences = row[4]

    if old_sentences != None:
        while True:
            old_sentences = old_sentences.replace(';', '; ')
            out = re.sub(r'(?<!\S)(\S+(?:\s\S+)*)\s+\1(?!\S)', '\\1', old_sentences)
            if out == old_sentences:
                break
            old_sentences = out

    new_sentences_eg.append(out)

temp_df_data = []
for i in range(len(vocabs)):
    temp_df_data.append([vocabs[i], new_def[i], new_sentences_eg[i]])

In [33]:
new_def_sent_df = spark.createDataFrame(temp_df_data, ["vocab", "new_def", "new_sentences"])

df = df.select('vocab', 'word_length', 'part_of_speech', 'synonyms', 'derived_words').join(new_def_sent_df, on='vocab')
df = df.withColumnRenamed('new_def', 'definitions').withColumnRenamed('new_sentences', 'sentences_eg')
df = df.select('vocab', 'word_length', 'part_of_speech', 'definitions', 'sentences_eg', 'synonyms', 'derived_words')

In [34]:
df = df.dropDuplicates()
df.show()

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|     sahaja|          6|    kata tugas|1. yg satu-satuny...|1. tuan sahaja yg...|satu-satunya;cuma...|bersahaja;bersaha...|
|      kerja|          5|     kata nama|1. usaha (kegiata...|1. beberapa mingg...|khidmat;pekerjaan...|sekerja;bekerja;m...|
|       kaum|          4|     kata nama|1. puak, suku ban...|1. banyak kaum or...|suku;kabilah;taef...|   berkaum;perkauman|
|      jelas|          5|      adjektif|1. terang (perkat...|1. suara itu tida...|terang;nyata;keta...|berjelas-jelas;me...|
|      waktu|          5|     kata nama|1. rangkaian atau...|1. secara rohania...|masa;jangka masa;...|sewaktu;sewaktu-w...|


## Write Processed Data into a CSV File

In [35]:
df.coalesce(1).write.csv('processed_data', header=True, mode='overwrite')

In [36]:
from python_script.hdfs_commander import *

run_hdfs_command('hdfs dfs -mv processed_data/part-00000-*.csv lexicon_data/processed_data.csv')
run_hdfs_command('hdfs dfs -rm -R processed_data')
run_hdfs_command('hdfs dfs -get lexicon_data/processed_data.csv lexicon_data/processed_data.csv')

Output: Deleted processed_data



## Terminate Spark Session

In [37]:
spark.stop()